# Selenium https://rollbit.com/ trade scrape

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install webdriver-manager
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas

In [4]:
import time
import datetime
import pandas as pd
from datetime import timezone
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

In [5]:
SLEEP = 0.5
ignored_exceptions = (StaleElementReferenceException,)
master_list = []

In [4]:
# TEST
import gspread_dataframe as gd
import gspread as gs
gc = gs.service_account(filename="your/cred/file.json")

def export_to_sheets(worksheet_name,df,mode='r'):
    ws = gc.open("SHEET_NAME").worksheet("worksheet_name")
    if(mode=='w'):
        ws.clear()
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=True,resize=True)
        return True
    elif(mode=='a'):
        ws.add_rows(df.shape[0])
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=False,row=ws.row_count+1,resize=False)
        return True
    else:
        return gd.get_as_dataframe(worksheet=ws)
    
df = pd.DataFrame.from_records([{'a': i, 'b': i * 2} for i in range(100)])
export_to_sheets("SHEET_NAME",df,'a')

ModuleNotFoundError: No module named 'gspread_dataframe'

In [6]:
def btn_click(xpath):
    try:        
        driver.execute_script("arguments[0].click();", WebDriverWait(
            driver, 1, SLEEP, ignored_exceptions=ignored_exceptions).until(
            EC.presence_of_element_located((By.XPATH, xpath))))
    except Exception as e:
        print(e)

In [7]:
def update(html):
    _up = '#72f238'
    dt = datetime.datetime.now(timezone.utc)
    time = dt.strftime('%Y-%m-%d-%H:%M:%S.%f')[:-3]

    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find('tbody').find_all('tr')

    for tr in trs:
        data = {}
        t = tr.find_all('td')
        
        data['time']         = time
        data['player']       = t[0].text
        data['trade']        = t[1].find('a')['href'].split('/')[2]
        _color               = t[1].find('svg', {'class': 'css-1pzuxf4'})['color']
        if _color == _up:
            data['position'] = 'UP'
        else:
            data['position'] = 'DOWN'
        data['wager']        = t[2].find('div').text
        data['multiplier']   = t[3].find('div').text
        data['pnl']          = t[4].find('div').text
        data['roi']          = t[5].find('div').text        

        print(data)
        master_list.append(data)
        break


    

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Lux\.wdm\drivers\chromedriver\win32\101.0.4951.41]
C:\Users\Lux\AppData\Local\Temp\ipykernel_2020\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
driver.get('https://rollbit.com')

In [10]:
_xpath_nav_trades = '//*[@id="root"]/div[6]/div[1]/div/div[5]/div[1]/div[4]'
btn_click(_xpath_nav_trades)

In [11]:
_c = 0
_prev = ''
_now = ''
_xpath_table = '//*[@id="root"]/div[6]/div[1]/div/div[5]/div[2]/div/table'

while 1:
    try:
        html_table = driver.find_element(By.XPATH, _xpath_table).get_attribute('innerHTML')
        _now = hash(html_table)
        if _prev != _now:
            _prev = _now
            update(html_table)
    except:
        print('Something went wrong, WebDown or InternetDown?')
    _c += 1
    if _c > 1800: 
        print('end')
        break
    time.sleep(0.1)

{'time': '2022-05-26-05:59:24.065', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '-$18.86', 'roi': '-47.15%'}
{'time': '2022-05-26-05:59:25.837', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '$40.91', 'roi': '102.27%'}
{'time': '2022-05-26-05:59:27.845', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '$3.32', 'roi': '8.30%'}
{'time': '2022-05-26-05:59:29.853', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '$27.38', 'roi': '68.45%'}
{'time': '2022-05-26-05:59:31.862', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '$21.00', 'roi': '52.50%'}
{'time': '2022-05-26-05:59:33.864', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'DOWN', 'w

In [12]:
df = pd.DataFrame(master_list)
df

,time,player,trade,position,wager,multiplier,pnl,roi
0,2022-05-26-05:59:24.065,boardo,Rollercoaster,DOWN,40.00,x600.00,-$18.86,-47.15%
1,2022-05-26-05:59:25.837,boardo,Rollercoaster,DOWN,40.00,x600.00,$40.91,102.27%
2,2022-05-26-05:59:27.845,boardo,Rollercoaster,DOWN,40.00,x600.00,$3.32,8.30%
3,2022-05-26-05:59:29.853,boardo,Rollercoaster,DOWN,40.00,x600.00,$27.38,68.45%
4,2022-05-26-05:59:31.862,boardo,Rollercoaster,DOWN,40.00,x600.00,$21.00,52.50%
5,2022-05-26-05:59:33.864,boardo,Rollercoaster,DOWN,40.00,x600.00,$15.22,38.05%
6,2022-05-26-05:59:35.371,boardo,Rollercoaster,DOWN,40.00,x600.00,-$18.86,-47.15%
7,2022-05-26-05:59:36.384,Richforever.,Rollercoaster,DOWN,20.00,x150.63,$32.48,162.40%
8,2022-05-26-05:59:46.888,boardo,Rollercoaster,UP,40.00,x600.00,-$18.80,-47.00%
9,2022-05-26-05:59:48.915,boardo,Rollercoaster,UP,40.00,x600.00,$24.26,60.65%


In [65]:
df.to_csv('./data/sample_rollbit.csv', index=False)